## Relay has not yet supported tail recursive optimization
some implemented optimizations:
- A-Normal Form
- Lambda Lift
- Inline Primitives
- Inliner
- Constant Pool Layout
- ADT Tag Allocation

In [1]:
import tvm
from tvm import relay
import numpy as np

## Relay has operators to get the shape of a tensor. 

In [2]:
def show_shape_of():
    a = relay.var('a', shape=(2, 2), dtype='float32')
    b = relay.var('b', shape=(2, 2), dtype='float32')
    c = relay.add(a, b)
    s = relay.shape_of(c)
    s0 = relay.take(s, relay.const(0, 'int32'))
    s1 = s0 + relay.const(1, 'int32')
    func = relay.Function([a, b], s1)
    mod = relay.Module()
    mod["main"] = func
    print(mod)
    
    for kind in ["debug", "vm"]:
        print('result in', kind)
        ex = relay.create_executor(kind, mod=mod, ctx=tvm.cpu(), target="llvm")
        result = ex.evaluate()(np.zeros((2, 2), dtype='float32'), np.zeros((2, 2), dtype='float32'))
        print(result.asnumpy())

In [3]:
show_shape_of()

v0.0.4
def @main(%a: Tensor[(2, 2), float32], %b: Tensor[(2, 2), float32]) -> int32 {
  %0 = add(%a, %b) /* ty=Tensor[(2, 2), float32] */;
  %1 = shape_of(%0, dtype="int32") /* ty=Tensor[(2), int32] */;
  %2 = take(%1, 0 /* ty=int32 */) /* ty=int32 */;
  add(%2, 1 /* ty=int32 */) /* ty=int32 */
}

result in debug
3
result in vm
3


## Does relay support operators like `unique`, `boolean_mask`?
Currently there is no Relay API like `relay.unique` or `relay.boolean_mask` but they are in progress. 

Some discussions about supporting dynamic shape:
- [Design problems for Relay to support NLP models](https://discuss.tvm.ai/t/design-problems-for-relay-to-support-nlp-models/1631/7)
- [Handling operators like `np.unique`, `boolean indexing` in relay](https://discuss.tvm.ai/t/handling-operators-like-np-unique-boolean-indexing-in-relay/1229)
- [[RFC][Relay] Dynamic Dimensions #3042](https://github.com/apache/incubator-tvm/issues/3042)

TVM v0.6 has already implemented type checking for Any. 

In [4]:
def any_dims(ndim):
    shape = []
    for _ in range(ndim):
        shape.append(relay.Any())
    return tuple(shape)

In [5]:
def show_any_broadcast(x_shape, y_shape, x_np_shape, y_np_shape, op, np_op):
    dtype = 'float32'
    x = relay.var('x', shape=x_shape, dtype=dtype)
    y = relay.var('y', shape=y_shape, dtype=dtype)
    mod = relay.module.Module()
    mod["main"] = relay.Function([x, y], op(x, y))
    print(mod)
    
    x_np = np.random.uniform(size=x_np_shape).astype(dtype)
    y_np = np.random.uniform(size=y_np_shape).astype(dtype)
    res_np = np_op(x_np, y_np)
    for kind in ["debug", "vm"]:
        print('result in', kind)
        ex = relay.create_executor(kind, mod=mod, ctx=tvm.cpu(), target="llvm")
        result = ex.evaluate()(x_np, y_np)
        print(result.asnumpy())
        tvm.testing.assert_allclose(result.asnumpy(), res_np)

In [6]:
show_any_broadcast((relay.Any(), 2), (1, 2), (1, 2), (1, 2), relay.add, np.add)

v0.0.4
def @main(%x: Tensor[(?, 2), float32], %y: Tensor[(1, 2), float32]) -> Tensor[(?, 2), float32] {
  add(%x, %y) /* ty=Tensor[(?, 2), float32] */
}

result in debug
[[1.7161348 1.7575006]]
result in vm
[[1.7161348 1.7575006]]


In [7]:
def show_arange():
    m, n, k = relay.ShapeVar('m'), relay.ShapeVar('n'), relay.ShapeVar('k')
    x = relay.var('x', shape=(m.var, n.var, k.var), dtype='float32')
    y0 = relay.shape_of(x)
    y1 = relay.take(y0, relay.const(0, 'int32'))
    y2 = relay.op.arange(y1, dtype='int32')
    y3 = y2 + relay.const(1, dtype='int32')
    mod = relay.module.Module()
    mod['main'] = relay.Function([x], y3, type_params=[m, n, k])
    print(mod)

    data = np.random.rand(10, 5, 3).astype('float32')
    for kind in ['debug', 'vm']:
        print('result in', kind)
        ex = relay.create_executor(kind, mod=mod, ctx=tvm.cpu(), target='llvm')
        result = ex.evaluate()(data)
        print(result.asnumpy())
        tvm.testing.assert_allclose(result.asnumpy(), np.array(range(10)).astype("int32")+1)

In [8]:
show_arange()

v0.0.4
def @main[m, n, k](%x: Tensor[(meta[Variable][0], meta[Variable][1], meta[Variable][2]), float32]) -> Tensor[(?), int32] {
  %0 = shape_of(%x, dtype="int32") /* ty=Tensor[(3), int32] */;
  %1 = take(%0, 0 /* ty=int32 */) /* ty=int32 */;
  %2 = arange(0 /* ty=int32 */, %1, 1 /* ty=int32 */, start=meta[relay.Constant][0], stop=meta[relay.Call][0], step=meta[relay.Constant][1], dtype="int32") /* ty=Tensor[(?), int32] */;
  add(%2, 1 /* ty=int32 */) /* ty=Tensor[(?), int32] */
}

// meta data omitted. you can use show_meta_data=True to include meta data
result in debug
[ 1  2  3  4  5  6  7  8  9 10]
result in vm
[ 1  2  3  4  5  6  7  8  9 10]
